In [16]:
import pandas as pd
import numpy as np

import os 

In this lab we will download and explore some data on soccer players.

In [17]:
# OR download from the disk into a data frame
os.getcwd()
os.path.expanduser("~/data")
os.chdir(os.path.expanduser("~/data"))
soccer = pd.read_csv("soccer.csv")

Let's take a look at the data.

The shape of the data set is: 

In [18]:
soccer.shape

(10, 8)

Column data types:

In [19]:
soccer.dtypes

PLAYER     object
SALARY     object
GP        float64
G           int64
A         float64
SOT         int64
PPG       float64
P         float64
dtype: object

The first 3 rows:

In [20]:
soccer.head(3)

,PLAYER,SALARY,GP,G,A,SOT,PPG,P
0,Sergio Agüero\n Forward — Manchester City,$19.2m,16.0,14,3.0,34,13.12,209.98
1,Eden Hazard\n Midfield — Chelsea,$18.9m,21.0,8,4.0,17,13.05,274.04
2,Alexis Sánchez\n Forward — Arsenal,$17.6m,NaN,12,7.0,29,11.19,223.86


__'GP'__ column has the following unique values:

In [21]:
soccer["GP"].unique()

array([ 16.,  21.,  nan,  18.,  13.,  20.,  15.])

Here is the concise description of the numerical columns of a data set:

In [22]:
soccer.describe()

,GP,G,A,SOT,PPG,P
count,9.000000,10.000000,8.000000,10.000000,10.000000,9.000000
mean,18.222222,7.000000,4.000000,18.400000,10.383000,188.912222
std,2.905933,3.858612,4.598136,7.974961,1.985218,46.092090
min,13.000000,2.000000,0.000000,10.000000,7.020000,132.230000
25%,16.000000,4.250000,1.000000,11.500000,10.020000,150.010000
50%,20.000000,6.500000,2.500000,18.000000,10.410000,197.910000
75%,20.000000,8.750000,4.750000,20.000000,11.140000,209.980000
max,21.000000,14.000000,14.000000,34.000000,13.120000,274.040000


Display the __'PLAYER'__ column:

In [23]:
soccer[['PLAYER']] # returns a DataFrame 

,PLAYER
0,Sergio Agüero\n Forward — Manchester City
1,Eden Hazard\n Midfield — Chelsea
2,Alexis Sánchez\n Forward — Arsenal
3,Yaya Touré\n Midfield — Manchester City
4,Ángel Di María\n Midfield — Manchester United
5,Santiago Cazorla\n Midfield — Arsenal
6,David Silva\n Midfield — Manchester City
7,Cesc Fàbregas\n Midfield — Chelsea
8,Saido Berahino\n Forward — West Brom
9,Steven Gerrard\n Midfield — Liverpool


Now, print how much money Santiago Cazorla earns:

In [24]:
# Find a partiular value
print(soccer.at[2,'SALARY'])


$17.6m


### Data preparation and clean up

Notice that before doing any data analysiswe should prepare, or clean up the data.
In this case the process involves:
- Converting the column names into lower case and renaming the columns 
- Removing \$m from __'salary'__ column
- Processing the __'player'__ column: turn the text into lower case, split the column into the name, position and team columns
- Dealing with the missing values (NaNs)

#### Rename the columns

First, let's convert the names of the columns into lower case:

In [25]:
soccer.columns = [c.lower() for c in soccer.columns]

In [26]:
soccer.tail(3)

,player,salary,gp,g,a,sot,ppg,p
7,Cesc Fàbregas\n Midfield — Chelsea,$14.0m,20.0,2,14.0,10,10.47,209.49
8,Saido Berahino\n Forward — West Brom,$13.8m,21.0,9,0.0,20,7.02,147.43
9,Steven Gerrard\n Midfield — Liverpool,$13.8m,20.0,5,1.0,11,7.50,150.01


Next, we change the names of the columns in the following way:

In [27]:
soccer = soccer.rename(columns = { 
                        'p':'points',
                        'gp': 'games',
                        'sot':'shot_on_target',
                        'g':'goals',
                        'a':'assists',
                        'ppg':'points_per_game',})
soccer.tail(3)

,player,salary,games,goals,assists,shot_on_target,points_per_game,points
7,Cesc Fàbregas\n Midfield — Chelsea,$14.0m,20.0,2,14.0,10,10.47,209.49
8,Saido Berahino\n Forward — West Brom,$13.8m,21.0,9,0.0,20,7.02,147.43
9,Steven Gerrard\n Midfield — Liverpool,$13.8m,20.0,5,1.0,11,7.50,150.01


#### Remove '$m' from the __'salary'__ column:

_Hint: use ```apply()``` method to apply a function to a column_

In [28]:
soccer.salary = soccer.salary.apply(lambda x: x.strip('$m'))
soccer.head(2)

,player,salary,games,goals,assists,shot_on_target,points_per_game,points
0,Sergio Agüero\n Forward — Manchester City,19.2,16.0,14,3.0,34,13.12,209.98
1,Eden Hazard\n Midfield — Chelsea,18.9,21.0,8,4.0,17,13.05,274.04


__Processing 'player' column__: 

First, turn the text in the column into lower case.

In [29]:
soccer.player = soccer.player.apply(lambda x: x.lower())
soccer[['player']].head(2)

,player
0,sergio agüero\n forward — manchester city
1,eden hazard\n midfield — chelsea


To split the column __'player'__, first create two additional columns __'position'__ and __'team'__.

_Hint: add each column as an empty pd.Series_

In [30]:
soccer['position'] = pd.Series('',index = soccer.index)
soccer.insert(loc = 9, column= 'team', value = '')

In [31]:
soccer.head(2)

,player,salary,games,goals,assists,shot_on_target,points_per_game,points,position,team
0,sergio agüero\n forward — manchester city,19.2,16.0,14,3.0,34,13.12,209.98,,
1,eden hazard\n midfield — chelsea,18.9,21.0,8,4.0,17,13.05,274.04,,


To transform values in column __'player'__, we first split on '\n' and next on '-'.

_Hint: you might first write a function that splits the text on '\n' and '-' into three parts and then use ```.apply()``` to apply this function to a column __'player'__._

In [32]:
def clean(text):
    name,rest = text.split('\n')
    position, team = [x.strip() for x in rest.split(' — ')]
    return pd.Series([name, position, team])

soccer[['name','position','team']] = soccer.player.apply(clean)  

In [33]:
soccer.head(3)

,player,salary,games,goals,assists,shot_on_target,points_per_game,points,position,team,name
0,sergio agüero\n forward — manchester city,19.2,16.0,14,3.0,34,13.12,209.98,forward,manchester city,sergio agüero
1,eden hazard\n midfield — chelsea,18.9,21.0,8,4.0,17,13.05,274.04,midfield,chelsea,eden hazard
2,alexis sánchez\n forward — arsenal,17.6,NaN,12,7.0,29,11.19,223.86,forward,arsenal,alexis sánchez


Now, let's drop the player column and rename the column __'name'__ into __'player'__ 

In [34]:
soccer.drop(['player'], axis = 1, inplace = True)

In [35]:
soccer = soccer.rename(columns = {'name':'player'})

In [36]:
soccer.head(3)

,salary,games,goals,assists,shot_on_target,points_per_game,points,position,team,player
0,19.2,16.0,14,3.0,34,13.12,209.98,forward,manchester city,sergio agüero
1,18.9,21.0,8,4.0,17,13.05,274.04,midfield,chelsea,eden hazard
2,17.6,NaN,12,7.0,29,11.19,223.86,forward,arsenal,alexis sánchez


Next, we can move the column player in front (position 0).

In [37]:
player = soccer['player']
soccer.drop(['player'], axis = 1, inplace = True)
soccer.insert(0,'player',player)  

In [38]:
soccer.tail(2)

,player,salary,games,goals,assists,shot_on_target,points_per_game,points,position,team
8,saido berahino,13.8,21.0,9,0.0,20,7.02,147.43,forward,west brom
9,steven gerrard,13.8,20.0,5,1.0,11,7.50,150.01,midfield,liverpool


#### Dealing with Missing Values (aka NaNs)

First, print the number of rows with NaNs

_Hint: use ```.shape``` method to count rows and ```.dropna()``` to get a data frame without NaNs _

In [39]:
nans = soccer.shape[0]-soccer.dropna().shape[0]
print('There are %d rows with missing values' % nans)

There are 3 rows with missing values


Select the rows with missing values in __'assists'__ and __'games'__ columns.

In [40]:
soccer[soccer['assists'].isnull()|soccer['games'].isnull()]

,player,salary,games,goals,assists,shot_on_target,points_per_game,points,position,team
2,alexis sánchez,17.6,NaN,12,7.0,29,11.19,223.86,forward,arsenal
4,ángel di maría,15.0,13.0,3,NaN,13,10.17,132.23,midfield,manchester united
5,santiago cazorla,14.8,20.0,4,NaN,20,9.97,NaN,midfield,arsenal


Select rows with no missing values in these two columns

In [41]:
soccer[soccer['assists'].notnull()&soccer['games'].notnull()]

,player,salary,games,goals,assists,shot_on_target,points_per_game,points,position,team
0,sergio agüero,19.2,16.0,14,3.0,34,13.12,209.98,forward,manchester city
1,eden hazard,18.9,21.0,8,4.0,17,13.05,274.04,midfield,chelsea
3,yaya touré,16.6,18.0,7,1.0,19,10.99,197.91,midfield,manchester city
6,david silva,14.3,15.0,6,2.0,11,10.35,155.26,midfield,manchester city
7,cesc fàbregas,14.0,20.0,2,14.0,10,10.47,209.49,midfield,chelsea
8,saido berahino,13.8,21.0,9,0.0,20,7.02,147.43,forward,west brom
9,steven gerrard,13.8,20.0,5,1.0,11,7.50,150.01,midfield,liverpool


Find the mean number of games per player:
- Apply the standard ```mean()``` 
- Compare the result with the sum of all values in a column 'games' divided by the total number of rows.

Note how pandas treats missing values in arithmetic operations

In [42]:
mean_games = soccer.games.mean()
print("Mean games (option 1):", round(mean_games,1))

Mean games (option 1): 18.2


In [43]:
mean_games = soccer.games.sum()/soccer.shape[0]
print("Mean games (option 2):", round(mean_games,1))

Mean games (option 2): 16.4


#### Filling NaN Rows

Let's fill missing values in column 'games' with the number derived from dividing the __'points'__ by  __'points_per_game'__ 

First, find the index of a row where column __'games'__ has a missing value.

_Hint: you might consider using ```np.where()``` to get a numpy array with the number of an index or apply ```np.isnan()``` function to a data frame column_

In [44]:
ind = soccer['games'].index[soccer['games'].apply(np.isnan)] # one way to find the index
i = np.where(soccer['games'].isnull())[0] # another way
n_games = soccer.at[ind[0],'points']/(soccer.at[ind[0],'points_per_game'])
print("The approximate number of games for",soccer.at[i[0],'player'], "is: ",round(n_games,0))

The approximate number of games for alexis sánchez is:  20.0


Then fill the missing value with the number you just found:

In [45]:
soccer['games'].fillna(value = n_games//1, inplace = True)

In [46]:
soccer.loc[ind[0]]

player             alexis sánchez
salary                       17.6
games                          20
goals                          12
assists                         7
shot_on_target                 29
points_per_game             11.19
points                     223.86
position                  forward
team                      arsenal
Name: 2, dtype: object

Finally, let's fill in NaNs in __'assists'__ columns with zeros

In [400]:
soccer['assists'].fillna(value = 0, inplace = True)
soccer

,player,salary,games,goals,assists,shot_on_target,points_per_game,points,position,team
0,sergio agüero,19.2,16.0,14,3.0,34,13.12,209.98,forward,manchester city
1,eden hazard,18.9,21.0,8,4.0,17,13.05,274.04,midfield,chelsea
2,alexis sánchez,17.6,20.0,12,7.0,29,11.19,223.86,forward,arsenal
3,yaya touré,16.6,18.0,7,1.0,19,10.99,197.91,midfield,manchester city
4,ángel di maría,15.0,13.0,3,0.0,13,10.17,132.23,midfield,manchester united
5,santiago cazorla,14.8,20.0,4,0.0,20,9.97,NaN,midfield,arsenal
6,david silva,14.3,15.0,6,2.0,11,10.35,155.26,midfield,manchester city
7,cesc fàbregas,14.0,20.0,2,14.0,10,10.47,209.49,midfield,chelsea
8,saido berahino,13.8,21.0,9,0.0,20,7.02,147.43,forward,west brom
9,steven gerrard,13.8,20.0,5,1.0,11,7.50,150.01,midfield,liverpool
